# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter
df = pd.read_csv('/datasets/data.csv') 
#display(df.tail(10)) # смотрим хвост
display(df.head(10)) # смотрим голову
#df.info # получаем общую информацию по таблице
#print(df.columns) # проверяем заголовки столбцов
#print(df.isna().sum()) # проверяем количество пропуском и где они 
#print(df.isnull().sum()) # также проверяем количество пропуском и где они
#print(df['dob_years'].min()) смотрим минимальный возраст клиента
#print(df['dob_years'].max()) смотрим максимальный возраст клиента


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Обнаруженные проблемы:
- в столбце education есть как строчные так и заглавные значения
- отрицательные значения в днях стажа
- одинаковое количество пропусков в days_employed и total_income
- в столбцах days_employed и total_income данные типо float, а должны быть int, поскольку маловероятно есть необходимость указывать в данных столбцах данные до ста тысячных долей
- минимальный значение в возрасте клиента = 0
- неизвестна валюта ежемесячного дохода

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df.sort_values(by = 'days_employed', ascending = False).tail(20) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21308,0,NaN,48,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья для сдачи
21311,0,NaN,49,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для сдачи
21321,0,NaN,56,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью
21350,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,на покупку автомобиля
21364,0,NaN,50,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,операции со своей недвижимостью
21369,2,NaN,42,среднее,1,в разводе,3,M,компаньон,0,NaN,покупка жилой недвижимости
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21391,0,NaN,52,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья для семьи
21407,1,NaN,36,среднее,1,женат / замужем,0,F,компаньон,0,NaN,строительство жилой недвижимости
21414,0,NaN,65,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка своего жилья


**Вывод**

одинаковое количество пропусков в days_employed и total_income, поскольку люди в этих столбцах старше 18 лет, маловероятно, что все они нигде не работали и не получали никакого дохода, следовательно скорей всего тут присутствует ошибка ввода данных.
заменим пропуски на нули, поскольку замена значений на медианные может повысить вероятность получения кредита.

Также обнаружены отрицательные значения в столбце days_employed- общий трудовой стаж, скорее всего ошибка внесения , преобразуем их в положительные

In [3]:
df['days_employed'] = df['days_employed'].apply(abs)

In [4]:
print(df['days_employed'].median())
print(df['total_income'].median())

2194.220566878695
145017.93753253992


также проверим максимальный трудовой стаж

In [5]:
df.sort_values(by = 'days_employed', ascending = False).head(15) #  сортируем столбец по убыванию

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


присутствие в данных стажа в тысячу лет, выглядит подозрительно, возможно присутствует ошибка в записи данных

In [6]:
df.loc[df['days_employed'] > 25550, 'days_employed'] = df['days_employed'].median()
df.sort_values(by = 'days_employed', ascending = False).head(15)
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [7]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) # заменяем пропущенные значения в столбце days_employed(отработанные дни)
# на 0
#df['total_income'] = df['total_income'].fillna(df['total_income'].median()) # заменяем пропущенные значения в столбце total_income — ежемесячный доход
# на 0
print(df.isnull().sum()) # проверяем смотрим количество пропушенных значений и где они находятся
# print(df.info()) # еще раз проверяеv


children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [8]:
display(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [9]:
#display('пенсионер',df.loc[df['income_type'] == 'пенсионер', 'total_income'].median())
#display('сотрудник',df.loc[df['income_type'] == 'сотрудник', 'total_income'].median())

for _type in df['income_type'].unique():
    median = df.loc[df['income_type'] == _type, 'total_income'].median()
    print('Медианная зарплата {} {:.2f}'.format(_type, median))
    df.loc[(df['total_income'].isna()) & (df['income_type'] == _type), 'total_income'] = median

Медианная зарплата сотрудник 142594.40
Медианная зарплата пенсионер 118514.49
Медианная зарплата компаньон 172357.95
Медианная зарплата госслужащий 150447.94
Медианная зарплата безработный 131339.75
Медианная зарплата предприниматель 499163.14
Медианная зарплата студент 98201.63
Медианная зарплата в декрете 53829.13


In [11]:
#df.sort_values(by = 'days_employed', ascending = False).tail(15)
df.sort_values(by = 'total_income', ascending = False).tail(10) 
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [12]:
# найдем самого возрастного человека и предположим, что он работает с 18 лет
max_days_of_employment = (df['dob_years'].max() - 18) * 365
max_days_of_employment # Получим количество дней, по которым можно сделать отсев

20805

In [13]:
df[df['days_employed'] > max_days_of_employment]['days_employed'].count() / 20805 * 100 # Многовато, почти 16.5% от всего набора
#переведем данные в года и добавим новый столбец 

0.0

In [14]:
df['years_employed'] = df['days_employed'] / 365

In [15]:
df['years_employed'] = df['years_employed'].astype(int)

In [16]:
del df['days_employed']# удалим столбец с днями и оставим только с года, работующий по тысяче лет отложим до лемматизации

In [17]:
df['children'].unique() # проверим столбец с детьми

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [18]:
# присутствуют необычные значение: -1 и 20, проверим сколько строк с -1
df[df['children'] == -1]['children'].count()

47

In [19]:
df['children'] = df['children'].replace(-1, 1) # проведем замену -1 на 1
df[df['children'] == -1]['children'].count() # проверим замену

0

In [20]:
df[df['children'] == 20]['children'].count()# проверим сколько строк с 20 детьми

76

In [21]:
df['children'].value_counts()# посмотрим соотношение этих строк к остальным знаениям 

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [22]:
# значение сильно выделяется из общих показателей, следовательно скорей всего ошибка в записи, поменяем 20 на 2
df['children'] = df['children'].replace(20, 2)
df['children'].unique()# проверяем замену

array([1, 0, 3, 2, 4, 5])

In [23]:
df['dob_years'].describe() # проверим возраст

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [24]:
df['dob_years'].value_counts()#проверия количество людей младше 18 лет, таких 101 значение в категории 0 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [25]:
df[df['years_employed'] > 70].groupby(['dob_years'])['dob_years'].count()

Series([], Name: dob_years, dtype: int64)

**Вывод**

Заменили пропущенные данные столбцов days_employed — общий трудовой стаж в днях и total_income — ежемесячный доход на 0, заменили стаж с дней на года и исправили неправильно записанное количество детей

### Замена типа данных

In [26]:
df['years_employed'] = df['years_employed'].astype(int) # меняем в столбце df['years_employed'] 
#вещественый тип float64 на int64
df['total_income'] = df['total_income'].astype(int) # меняем в столбце data['df_employed']
print(df.info())
#df[df.duplicated(keep = False)].sort_values(by = data.columns.values.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
years_employed      21525 non-null int64
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


**Вывод**

В столбцах years_employed и total_income содержаться данные вещественного типа (float), а должны быть целочисленные тк врядли зарплата указывается до ста тысячных долей рубля и отработанные дни тоже использован метод astype

### Обработка дубликатов

Как мы уже заметили в столбцах 'education' и 'family_status' гуляет регистр, приведем его к нижнему регистру

In [27]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [28]:
display(df['education'].value_counts())
print('__________________________________')
display(df['family_status'].value_counts())#проверия изменения

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

__________________________________


женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [29]:
df.duplicated().sum() # проверим наличие дублей

71

In [30]:
df = df.drop_duplicates().reset_index(drop=True) # Удалим дубли и сбросим индекс
df.duplicated().sum() # Проверим удалились ли дубли

0

**Вывод**

Нашли дубликаты и удалили их

### Лемматизация

In [31]:
#импортируем библиотеки
m = Mystem()
lemmas = m.lemmatize(df['purpose'][3])
print(lemmas)
print(len(df['purpose']))

['дополнительный', ' ', 'образование', '\n']
21454


In [32]:
uniq_purpose = df['purpose'].unique()
print(uniq_purpose) #берем только уникальные значения

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [33]:
uniq_purpose = uniq_purpose.tolist()
print(uniq_purpose) #преобразуем массив в питоновский лист

['покупка жилья', 'приобретение автомобиля', 'дополнительное образование', 'сыграть свадьбу', 'операции с жильем', 'образование', 'на проведение свадьбы', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'автомобили', 'заняться образованием', 'сделка с подержанным автомобилем', 'получение образования', 'автомобиль', 'свадьба', 'получение дополнительного образования', 'покупка своего жилья', 'операции с недвижимостью', 'получение высшего образования', 'свой автомобиль', 'сделка с автомобилем', 'профильное образование', 'высшее образование', 'покупка жилья для сдачи', 'на покупку автомобиля', 'ремонт жилью', 'заняться высшим образованием']


In [34]:
uniq_purpose_lemm = []
for i in uniq_purpose:
    uniq_purpose_lemm.extend(m.lemmatize(i))
    print(uniq_purpose_lemm) #Функция для Лемматизации элементов

['покупка', ' ', 'жилье', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n']
['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ',

In [35]:
print(Counter(uniq_purpose_lemm)) # подсчитываем количество упоминаний в тексте

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


чаще всего встречается: образование, недвижимость, автомобиль, жилье, свадьба

In [36]:
def lemm_purpose(row):
    if 'недвижимость' in m.lemmatize(row):
        return 'недвижимость'
           
    if 'автомобиль' in m.lemmatize(row):
        return 'автомобиль'
            
    if 'жилье' in m.lemmatize(row):
        return 'жилье'
            
    if 'свадьба' in m.lemmatize(row):
        return 'свадьба'
    
    if 'образование' in m.lemmatize(row):
        return 'образование'
             
    else: return 'другое'

In [37]:
df['purpose'] = df['purpose'].apply(lemm_purpose)
display(df.head()) # применяем к столбцу data['purpose'] и проверяем

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,11
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,6


**Вывод**

Данные столбца data['purpose'] лемматизированы, цели для удобства сортировки отражены одним словом

### Категоризация данных

Делаем сводную таблицу: влияние количество детей клиента на факт погашения кредита
Bстроим строку с процентами кто имел задолжность количество детей клиента факт погашения кредита

In [38]:
df_child = df.pivot_table(index = ['children'], values = 'debt', aggfunc = [sum, 'count'])
#df_child = df.groupby(['children',]).agg({'debt':'sum','children':'count'})
df_child['%_total_debt'] = (df_child ['sum','debt']/df_child ['count','debt'])
#df_child.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_child.style.format({'% невозврата':'{:.2%}'})
#print(df_child)

,sum,count,%_total_debt
,debt,debt,
children,,,
0,1063,14091,0.0754382
1,445,4855,0.0916581
2,202,2128,0.0949248
3,27,330,0.0818182
4,4,41,0.097561
5,0,9,0


Делаем сводную таблицу *семейное положение * - факт погашения кредита Bстроим строку с процентами кто имел задолжность *семейное положение * - факт погашения кредита

In [39]:
df_family = df.groupby('family_status').agg({'debt':'sum', 'family_status':'count'})
df_family['%_total_debt'] = (df_family['debt']/df_family['family_status'])
df_family.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_family.style.format({'% невозврата':'{:.2%}'})
#print(df_family)

,Кол-во невозвратов,Кол-во заемщиков,% невозврата
family_status,,,
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%
не женат / не замужем,274,2810,9.75%


Сделаем общую сводную таблицу c процентами

In [40]:
df_child_family = df.groupby(['children', 'family_status']).agg({'debt':'sum', 'family_status':'count'})
df_child_family['%_total_debt'] = (df_child_family['debt']/df_child_family['family_status'])*100
df_child_family.columns = ['Кол. невозвр','Кол-во заемщиков','% невозврата']
#df_child_family.format({'Процент невозврата':'{:.2%}'})
display(df_child_family)

Кол. невозвр  Кол-во заемщиков  % невозврата
children family_status                                                      
0        в разводе                        55               784      7.015306
         вдовец / вдова                   53               847      6.257379
         гражданский брак                229              2730      8.388278
         женат / замужем                 516              7468      6.909480
         не женат / не замужем           210              2262      9.283820
1        в разводе                        21               316      6.645570
         вдовец / вдова                    7                81      8.641975
         гражданский брак                118              1000     11.800000
         женат / замужем                 247              3004      8.222370
         не женат / не замужем            52               454     11.453744
2        в разводе                         8                83      9.638554
         вдовец / вдова                    3                24     12.500000
         гражданский брак                 33               355      9.295775
         женат / замужем                 148              1582      9.355247
         не женат / не замужем            10                84     11.904762
3        в разводе                         1                11      9.090909
         вдовец / вдова                    0                 6      0.000000
         гражданский брак                  8                56     14.285714
         женат / замужем                  17               249      6.827309
         не женат / не замужем             1                 8     12.500000
4        в разводе                         0                 1      0.000000
         вдовец / вдова                    0                 1      0.000000
         гражданский брак                  0                 8      0.000000
         женат / замужем                   3                29     10.344828
         не женат / не замужем             1                 2     50.000000
5        гражданский брак                  0                 2      0.000000
         женат / замужем                   0                 7      0.000000

Проверим как разные цели кредита влияют на его возврат в срок?

In [41]:
df_goal = df.pivot_table(index = ['purpose'], values = 'debt', aggfunc = [sum, 'count'])
#df_child = df.groupby(['children',]).agg({'debt':'sum','children':'count'})
df_goal['%_total_debt'] = (df_goal['sum','debt']/df_goal['count','debt'])
df_goal.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_goal.style.format({'% невозврата':'{:.2%}'})
#print(df_child)

,Кол-во невозвратов,Кол-во заемщиков,% невозврата
purpose,,,
автомобиль,403,4306,9.36%
жилье,308,4460,6.91%
недвижимость,474,6351,7.46%
образование,370,4013,9.22%
свадьба,186,2324,8.00%


**Вывод**

Сводные таблицы сделаны данные представлены в удобном для чтения виде

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
df_child = df.pivot_table(index = ['children'], values = 'debt', aggfunc = [sum, 'count'])
df_child['%_total_debt'] = (df_child ['sum','debt']/df_child ['count','debt'])
df_child.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_child.style.format({'% невозврата':'{:.2%}'})

,Кол-во невозвратов,Кол-во заемщиков,% невозврата
children,,,
0,1063,14091,7.54%
1,445,4855,9.17%
2,202,2128,9.49%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


**Вывод**

Условная зависимость присутствует, люди без детей чаще погашают кредиты в срок, но разница очень мала, в следствие чего присутствует вероятность статистической погрешности

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
df_family = df.groupby('family_status').agg({'debt':'sum', 'family_status':'count'})
df_family['%_total_debt'] = (df_family['debt']/df_family['family_status'])
df_family.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_family.style.format({'% невозврата':'{:.2%}'})

,Кол-во невозвратов,Кол-во заемщиков,% невозврата
family_status,,,
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%
не женат / не замужем,274,2810,9.75%


**Вывод**

Самые низкие показатели у показателей 'в разводе' и 'вдовец / вдова'  можно предположить, что погашение кредитов зависит в том числе от наличия пары в семье.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

сгруппируем людей по уровню доходов в 4 группы

In [44]:
df['total_income'].quantile([0.25,0.5,0.75])

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64

In [45]:
def income_status(total_income):
    if total_income <= 88946.5:
            return 'Низкий уровень дохода'
    if total_income <= 135716.0:
            return 'Средний уровень дохода'
    if total_income < 195751.5:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'

In [46]:
df['income_status'] = df['total_income'].apply(income_status)

In [47]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,23,Сверхвысокий уровень дохода
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,11,Средний уровень дохода
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,15,Высокий уровень дохода
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,11,Сверхвысокий уровень дохода
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,6,Высокий уровень дохода


In [48]:
total_income_set = df.groupby('income_status')['debt'].sum() / \
      df.groupby('income_status')['debt'].count() * 100

In [49]:
total_income_set.sort_values()

income_status
Сверхвысокий уровень дохода    7.133544
Низкий уровень дохода          7.666256
Средний уровень дохода         8.515815
Высокий уровень дохода         8.739235
Name: debt, dtype: float64

In [50]:
total_income_pivot = df.pivot_table(index=['income_status'], columns=['debt'], values='education_id', aggfunc='count')
total_income_pivot['ratio'] = total_income_pivot[1] / total_income_pivot[0] * 100
total_income_pivot

debt,0,1,ratio
income_status,,,
Высокий уровень дохода,6464,619,9.576114
Низкий уровень дохода,2999,249,8.302768
Сверхвысокий уровень дохода,4986,383,7.681508
Средний уровень дохода,5264,490,9.308511


In [51]:
# Проверим количество людей в каждой из групп
df['income_status'].value_counts(normalize = True)

Высокий уровень дохода         0.330148
Средний уровень дохода         0.268202
Сверхвысокий уровень дохода    0.250256
Низкий уровень дохода          0.151394
Name: income_status, dtype: float64

**Вывод**

Самые плохие показатели у групп со средним и высоким доходом, 
С количеством человек в каждой из групп — порядок.

- Как разные цели кредита влияют на его возврат в срок?

In [52]:
df_goal = df.pivot_table(index = ['purpose'], values = 'debt', aggfunc = [sum, 'count'])
#df_child = df.groupby(['children',]).agg({'debt':'sum','children':'count'})
df_goal['%_total_debt'] = (df_goal['sum','debt']/df_goal['count','debt'])
df_goal.columns = ['Кол-во невозвратов','Кол-во заемщиков','% невозврата']
df_goal.style.format({'% невозврата':'{:.2%}'})
#print(df_child)


,Кол-во невозвратов,Кол-во заемщиков,% невозврата
purpose,,,
автомобиль,403,4306,9.36%
жилье,308,4460,6.91%
недвижимость,474,6351,7.46%
образование,370,4013,9.22%
свадьба,186,2324,8.00%


**Вывод**

Хуже всего выглядят категории: образование и автомобиль. 

## Шаг 4. Общий вывод

Присутствует зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье. Нельзя сказать, что они кардинально отличаются, но они   могут оказать существенное влияние на кредитный портфель банка. 

# Рекомендации для банка
- Продумать сценарий, когда отличается дата или время заявки, а уникальный идентификатор пользователя один и тот же, 
  иначе фактически дубли будут, но метод duplicated() их не найдет.
- Уделить внимание правилам валидации на сервере, чтобы больше не сталкиваться с проблемой дублей.  
- Блокировать отправку заявок с заведомо некорректными значениями, которые могут быть важны для анализа, например,
  с отрицательным количеством детей.
- Считать стаж работы в годах, пользователи чаще всего не знают свой стаж в днях.